## 第9章について
とりあえずこのipynbでは、上から400行くらい取ってきたminimunデータで実験しています。<br>
1/00サンプリングでも重すぎるので。。。

In [1]:
#coding:utf-8

import csv
import codecs
from collections import Counter,OrderedDict
from itertools import chain
import random
import numpy as np
from nltk import tokenize
import os
import sklearn.decomposition
# from sklearn.decomposition import PCA
import pickle
from scipy import sparse, io
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
import re
import pandas as pd
import scipy.spatial.distance as dis


In [2]:
# データをロード
input_name = './sample.txt'
f = codecs.open(input_name, "r","utf-8")
wiki_data = str(f.read()).decode("utf-8")
f.close
wiki_data

u'Anarchism\n\nAnarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions, but that several authors have defined as more specific institutions based on non-hierarchical free associations. Anarchism holds the state to be undesirable, unnecessary, or harmful. While anti-statism is central, anarchism entails opposing authority or hierarchical organisation in the conduct of human relations, including, but not limited to, the state system.\nAs a subtle and anti-dogmatic philosophy, anarchism draws on many currents of thought and strategy. Anarchism does not offer a fixed body of doctrine from a single particular world view, instead fluxing and flowing as a philosophy. There are many types and traditions of anarchism, not all of which are mutually exclusive. Anarchist schools of thought can differ fundamentally, supporting anything from extreme individualism to complete collectivism. Strains of anarchism have often been divide

## 80. コーパスの整形
    [問題]
    文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである．
    ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう．
    そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．
    トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
    空文字列となったトークンは削除<br>
	以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．
    
[参照]<br>
    => http://qiita.com/segavvy/items/ea485e66dd96eee891da



In [3]:
word_list=[]
for chunk in wiki_data.split(' '):
	token = chunk.strip('.,!?;:()[]\'"').strip()
	if len(token) > 0:
		word_list.append(str(token))
word_list

['Anarchism\n\nAnarchism',
 'is',
 'a',
 'political',
 'philosophy',
 'that',
 'advocates',
 'stateless',
 'societies',
 'often',
 'defined',
 'as',
 'self-governed',
 'voluntary',
 'institutions',
 'but',
 'that',
 'several',
 'authors',
 'have',
 'defined',
 'as',
 'more',
 'specific',
 'institutions',
 'based',
 'on',
 'non-hierarchical',
 'free',
 'associations',
 'Anarchism',
 'holds',
 'the',
 'state',
 'to',
 'be',
 'undesirable',
 'unnecessary',
 'or',
 'harmful',
 'While',
 'anti-statism',
 'is',
 'central',
 'anarchism',
 'entails',
 'opposing',
 'authority',
 'or',
 'hierarchical',
 'organisation',
 'in',
 'the',
 'conduct',
 'of',
 'human',
 'relations',
 'including',
 'but',
 'not',
 'limited',
 'to',
 'the',
 'state',
 'system.\nAs',
 'a',
 'subtle',
 'and',
 'anti-dogmatic',
 'philosophy',
 'anarchism',
 'draws',
 'on',
 'many',
 'currents',
 'of',
 'thought',
 'and',
 'strategy',
 'Anarchism',
 'does',
 'not',
 'offer',
 'a',
 'fixed',
 'body',
 'of',
 'doctrine',
 'fro

In [4]:
output_str = ""
for word in word_list:
	output_str = output_str + " " + word
output_str

' Anarchism\n\nAnarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system.\nAs a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the catego

In [5]:
output_name = './80_sample.txt'
f = open(output_name,"w")
f.write(output_str)
f.close

<function close>

### 余談
	以下は正規表現でやろうとして失敗。誰か教えてくださいw

```
pattern = re.compile(".,!?;:()\[]")
for tokenized_word in tokenize.word_tokenize(wiki_data):
	word_list.append(re.sub(pattern, "", str(tokenized_word)))
```

## 81. 複合語からなる国名への対処
    [問題]
    英語では，複数の語の連接が意味を成すことがある．
    例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．
    そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．
    しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

    インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．
    例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．


In [11]:
country_name_file = "./country_name.txt"
f = open(country_name_file,"r")
countries = f.read()
f.close()

# 一単語目をkey、単語全体をvalueとして入れている
name_set=[]
country_name_dic = {}
for name in countries.split("\n"):
	splited_name = name.split(" ")
	if len(splited_name) > 1:
		name_set.append(name)
		if not country_name_dic.has_key(splited_name[0]):
			country_name_dic.update({splited_name[0]:[len(splited_name)]})
		else:
			country_name_dic[splited_name[0]].extend([len(splited_name)])
print "以下を単語辞書としてマッチさせる\n"
name_set

['American Samoa',
 'Antigua and Barbuda',
 'Ashmore and Cartier Islands',
 'Bahamas, The',
 'Bassas da India',
 'Bosnia and Herzegovina',
 'Bouvet Island',
 'British Indian Ocean Territory',
 'British Virgin Islands',
 'Burkina Faso',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Christmas Island',
 'Clipperton Island',
 'Cocos (Keeling) Islands',
 'Congo, Democratic Republic of the',
 'Congo, Republic of the',
 'Cook Islands',
 'Coral Sea Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Czech Republic',
 'Dominican Republic',
 'El Salvador',
 'Equatorial Guinea',
 'Europa Island',
 'Falkland Islands (Islas Malvinas)',
 'Faroe Islands',
 'French Guiana',
 'French Polynesia',
 'French Southern and Antarctic Lands',
 'Gambia, The',
 'Gaza Strip',
 'Glorioso Islands',
 'Heard Island and McDonald Islands',
 'Holy See (Vatican City)',
 'Hong Kong',
 'Isle of Man',
 'Jan Mayen',
 'Juan de Nova Island',
 'Korea, North',
 'Korea, South',
 'Marshall Islands',
 'Micronesia, Feder

In [12]:
country_name_dic

{'American': [2],
 'Antigua': [3],
 'Ashmore': [4],
 'Bahamas,': [2],
 'Bassas': [3],
 'Bosnia': [3],
 'Bouvet': [2],
 'British': [4, 3],
 'Burkina': [2],
 'Cape': [2],
 'Cayman': [2],
 'Central': [3],
 'Christmas': [2],
 'Clipperton': [2],
 'Cocos': [3],
 'Congo,': [5, 4],
 'Cook': [2],
 'Coral': [3],
 'Costa': [2],
 'Cote': [2],
 'Czech': [2],
 'Dominican': [2],
 'El': [2],
 'Equatorial': [2],
 'Europa': [2],
 'Falkland': [4],
 'Faroe': [2],
 'French': [2, 2, 5],
 'Gambia,': [2],
 'Gaza': [2],
 'Glorioso': [2],
 'Heard': [5],
 'Holy': [4],
 'Hong': [2],
 'Isle': [3],
 'Jan': [2],
 'Juan': [4],
 'Korea,': [2, 2],
 'Marshall': [2],
 'Micronesia,': [4],
 'Navassa': [2],
 'Netherlands': [2],
 'New': [2, 2],
 'Norfolk': [2],
 'Northern': [3],
 'Papua': [3],
 'Paracel': [2],
 'Pitcairn': [2],
 'Puerto': [2],
 'Saint': [2, 4, 2, 4, 5],
 'San': [2],
 'Sao': [4],
 'Saudi': [2],
 'Serbia': [3],
 'Sierra': [2],
 'Solomon': [2],
 'South': [2, 7],
 'Spratly': [2],
 'Sri': [2],
 'Trinidad': [3],
 

In [17]:
# ここから、置換作業
g = open("./80_sample.txt","r")
_80_data = g.readlines()
g.close()

h = open("./81_sample.txt","w")
for line in _80_data:
	token_list = line.split(" ")
	result = []
	for i,token in enumerate(token_list):
		if not token in country_name_dic.keys():
			result.append(token_list[i])
		else:
			for length in country_name_dic[token]:
				if " ".join(token_list[i : i+length]) in name_set:
					result.append("_".join(token_list[i : i+length]))
					i =i+length
					# 二つ目の単語が混入するバグがあり
	print result
	h.write(" ".join(result[0:len(result)]))
h.close()

## 82. 文脈の抽出
    [問題]
    81で作成したコーパス中に出現するすべての単語tに関して，単語tと文脈語cのペアをタブ区切り形式ですべて書き出せ．
    ただし，文脈語の定義は次の通りとする．    
    ある単語tの前後d単語を文脈語cとして抽出する（ただし，文脈語に単語tそのものは含まない）単語tを選ぶ度に，文脈幅dは{1,2,3,4,5}の範囲でランダムに決める．
    
    [方針]
    つまり、全ての単語に対して、文脈幅dをランダムに選んで表示するという問題?
    
    # ランダムの理由を調べる、最適化の計算量的な問題かも

In [50]:
f = open("./80_sample.txt","r")
wiki_data = f.read()

# リストにして配列番号で処理
splited_data = wiki_data.split(" ")

output_string = ""
for target_index,target_word in enumerate(splited_data):

	plut_count = random.randint(1,5)
	minus_count = 0 - int(plut_count)

	for i in range(minus_count,plut_count):
		if i != 0:
			context_index = int(target_index)+int(i)

			try:
				tab_separted_context = target_word+"\t"+splited_data[context_index]+"\n"
				output_string = output_string + tab_separted_context
				print target_word,"\t",splited_data[context_index]
			except:
				print context_index,"nan"
output_string

'\tof\n\tThurgot\'s\n\twould-be\n\tsucce\n\tAnarchism\n\nAnarchism\n\tis\n\ta\nAnarchism\n\nAnarchism\tThurgot\'s\nAnarchism\n\nAnarchism\twould-be\nAnarchism\n\nAnarchism\tsucce\nAnarchism\n\nAnarchism\t\nAnarchism\n\nAnarchism\tis\nAnarchism\n\nAnarchism\ta\nAnarchism\n\nAnarchism\tpolitical\nis\tsucce\nis\t\nis\tAnarchism\n\nAnarchism\nis\ta\nis\tpolitical\na\tAnarchism\n\nAnarchism\na\tis\na\tpolitical\npolitical\ta\nphilosophy\t\nphilosophy\tAnarchism\n\nAnarchism\nphilosophy\tis\nphilosophy\ta\nphilosophy\tpolitical\nphilosophy\tthat\nphilosophy\tadvocates\nphilosophy\tstateless\nphilosophy\tsocieties\nthat\ta\nthat\tpolitical\nthat\tphilosophy\nthat\tadvocates\nthat\tstateless\nadvocates\tthat\nstateless\tpolitical\nstateless\tphilosophy\nstateless\tthat\nstateless\tadvocates\nstateless\tsocieties\nstateless\toften\nstateless\tdefined\nsocieties\tpolitical\nsocieties\tphilosophy\nsocieties\tthat\nsocieties\tadvocates\nsocieties\tstateless\nsocieties\toften\nsocieties\tdefined\ns

In [20]:
output_file = "82_sample.txt"
g = open(output_file,"w")
g.write(output_string)

# Alexanderなどバグ取りがまだ

##  83. 単語／文脈の頻度の計測
    [問題]
	82の出力を利用し，以下の出現分布，および定数を求めよ．
    f(t,c):	単語tと文脈語cの共起回数
    f(t,∗):	単語tの出現回数
	f(∗,c):	文脈語cの出現回数
	N: 		単語と文脈語のペアの総出現回数

In [26]:
input_file = "./82_sample.txt"
f = open(input_file,"r")
tabbed_context = f.read()

In [27]:
tabbed_context

'\tcase\n\tof\n\tThurgot\'s\n\twould-be\n\tsucce\n\tAnarchism\n\nAnarchism\n\tis\n\ta\n\tpolitical\nAnarchism\n\nAnarchism\tsucce\nAnarchism\n\nAnarchism\t\nAnarchism\n\nAnarchism\tis\nis\tThurgot\'s\nis\twould-be\nis\tsucce\nis\t\nis\tAnarchism\n\nAnarchism\nis\ta\nis\tpolitical\nis\tphilosophy\nis\tthat\na\twould-be\na\tsucce\na\t\na\tAnarchism\n\nAnarchism\na\tis\na\tpolitical\na\tphilosophy\na\tthat\na\tadvocates\npolitical\tsucce\npolitical\t\npolitical\tAnarchism\n\nAnarchism\npolitical\tis\npolitical\ta\npolitical\tphilosophy\npolitical\tthat\npolitical\tadvocates\npolitical\tstateless\nphilosophy\tis\nphilosophy\ta\nphilosophy\tpolitical\nphilosophy\tthat\nphilosophy\tadvocates\nthat\ta\nthat\tpolitical\nthat\tphilosophy\nthat\tadvocates\nthat\tstateless\nadvocates\tis\nadvocates\ta\nadvocates\tpolitical\nadvocates\tphilosophy\nadvocates\tthat\nadvocates\tstateless\nadvocates\tsocieties\nadvocates\toften\nadvocates\tdefined\nstateless\tpolitical\nstateless\tphilosophy\nstateles

In [28]:
tabbed_list = []
for context in tabbed_context.split("\n"):
	context_tmp = [i.lower() for i in context.split("\t")]
	context_tmp = tuple(context_tmp)
	tabbed_list.append(context_tmp)
print len(tabbed_list),"=>",len(set(tabbed_list))
collocated_list = set(filter(lambda x: len(x)>1,tabbed_list))

In [29]:
collocated_list

{('national', 'revolution.'),
 ('form', 'or'),
 ('favour', 'capitalism'),
 ('which', 'literary'),
 ('architecture', 'stored'),
 ('by', 'influenced'),
 ('and', 'unmarried'),
 ('computer', 'larson'),
 ('by', 'uninfluenced'),
 ('series', 'when'),
 ('moderna', 'escuela'),
 ('the', 'waqf'),
 ('the', 'bear'),
 ('shah', 'is'),
 ('at', 'split'),
 ('pay', 'per'),
 ('republics', 'soviet'),
 ('for', 'made'),
 ('1,485,000', 'are'),
 ('several', 'industry'),
 ('is', 'free'),
 ('realm', 'i.e'),
 ('experimented', 'when'),
 ('problem', 'could'),
 ('among', 'is'),
 ('peasant', 'established'),
 ('modern', 'still'),
 ('revolutionary', 'seeing'),
 ('the', "workingmen's"),
 ('at', 'times'),
 ('great', 'becomes'),
 ('pervert', 'japanese'),
 ('more', '1980s'),
 ('new', '1911'),
 ('remarry', 'walter'),
 ('with', 'full'),
 ('such', 'themselves'),
 ('30', 'the'),
 ('issues', 'is'),
 ('in', 'up'),
 ('or', 'business'),
 ('over', 'those'),
 ('to', 'panic'),
 ('at', 'early'),
 ('any', 'animation'),
 ('exercise', 'f

In [31]:
frequency_list=[]
for collocation in collocated_list:
	# print collocation,"の組み合わせをチェックする"
	_search = collocation[0]+"\t"+collocation[1]
	tmp = [_search,tabbed_context.count(_search),tabbed_context.count(collocation[0]),tabbed_context.count(collocation[1])]
	frequency_list.append(tmp)

frequency_list

[['national\trevolution.', 0, 581, 0],
 ['form\tor', 1, 781, 9683],
 ['favour\tcapitalism', 1, 28, 69],
 ['which\tliterary', 1, 584, 54],
 ['architecture\tstored', 1, 13, 27],
 ['by\tinfluenced', 3, 1114, 85],
 ['and\tunmarried', 1, 6977, 14],
 ['computer\tlarson', 0, 221, 0],
 ['by\tuninfluenced', 1, 1114, 12],
 ['series\twhen', 1, 180, 155],
 ['moderna\tescuela', 0, 0, 0],
 ['the\twaqf', 2, 12668, 14],
 ['the\tbear', 0, 12668, 12],
 ['shah\tis', 0, 0, 12042],
 ['at\tsplit', 2, 10987, 23],
 ['pay\tper', 1, 58, 1150],
 ['republics\tsoviet', 0, 8, 0],
 ['for\tmade', 1, 2290, 89],
 ['1,485,000\tare', 1, 14, 960],
 ['several\tindustry', 1, 41, 174],
 ['is\tfree', 3, 12042, 391],
 ['realm\ti.e', 1, 11, 75],
 ['experimented\twhen', 1, 11, 155],
 ['problem\tcould', 1, 106, 77],
 ['among\tis', 1, 157, 12042],
 ['peasant\testablished', 1, 25, 130],
 ['modern\tstill', 1, 98, 85],
 ['revolutionary\tseeing', 1, 196, 24],
 ["the\tworkingmen's", 0, 12668, 0],
 ['at\ttimes', 1, 10987, 79],
 ['great\

In [32]:
output_file = "./83_sample.csv"
g = open(output_file,"w")
g.write(str(len(tabbed_list)))
g.write("\n")

writer=csv.writer(g)
for i in frequency_list:
    writer.writerow(i)
g.close

<function close>

##  84. 単語文脈行列の作成
    [問題]
	83の出力を利用し，単語文脈行列Xを作成せよ．ただし，行列Xの各要素Xtcは次のように定義する．
	f(t,c)≥10 ならば，Xtc=PPMI(t,c)=max{logN×f(t,c)f(t,∗)×f(∗,c),0}
	f(t,c)<10 ならば，Xtc=0
	ここで，PPMI(t,c)はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．
    なお，行列Xの行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．
    幸い，行列Xのほとんどの要素は0になるので，非0の要素だけを書き出せばよい．
    
    [方針]
    一部ここを参考にした
http://qiita.com/segavvy/items/21455b802e34a9e49f92d <br>
http://d.hatena.ne.jp/billest/20090906/1252269157


In [19]:
N = int(87867)

In [20]:
# 一行目を削除してからファイルを開いてください
input_file = "./83_sample.csv"
freq_df = pd.read_csv(input_file,header=None)
freq_df.columns = ["collocation","tc","t*","*c"]
freq_df

,collocation,tc,t*,*c
0,form\tor,1,762,10061
1,for\tparity,0,2297,27
2,favour\tcapitalism,1,36,86
3,was\tii.,0,1766,0
4,by\tinfluenced,3,1135,73
5,with\tiran,0,1471,0
6,in\tsame,0,15905,51
7,computer\tlarson,0,229,0
8,by\tuninfluenced,1,1135,9
9,and\tgraphics,1,7076,7


In [21]:
over10_df = freq_df.query("tc >= 10")
under10_df = freq_df.query("tc < 10")
len(freq_df),len(over10_df)

(60573, 818)

In [22]:
# PPMIを計算する関数を定義
def calc_PPMI(tc_list):
	N = int(87867)
	return max(np.log(N*tc_list[1]/tc_list[1]*tc_list[2]),0)

In [23]:
_dic = {}
for i,tc_row in over10_df.iterrows():
	_dic.update({tc_row[0]:{"ppmi":calc_PPMI(tc_row.tolist())}})
ppmi_df = pd.DataFrame.from_dict(_dic).T

In [24]:
ppmi_df

,ppmi
\ta,25.315567
\tanarchism,25.315567
\tdepicted,25.315567
\tfantasy,25.315567
\tin,25.315567
\tis,25.315567
\tpolitical,25.315567
1\ta,19.468450
1980s\tthe,15.701068
19th\tthe,15.701068


In [25]:
freq_df["term"] = freq_df["collocation"].str.split("\t").apply(lambda x: x[0])
freq_df["col"] = freq_df["collocation"].str.split("\t").apply(lambda x: x[1])
freq_df

,collocation,tc,t*,*c,term,col
0,form\tor,1,762,10061,form,or
1,for\tparity,0,2297,27,for,parity
2,favour\tcapitalism,1,36,86,favour,capitalism
3,was\tii.,0,1766,0,was,ii.
4,by\tinfluenced,3,1135,73,by,influenced
5,with\tiran,0,1471,0,with,iran
6,in\tsame,0,15905,51,in,same
7,computer\tlarson,0,229,0,computer,larson
8,by\tuninfluenced,1,1135,9,by,uninfluenced
9,and\tgraphics,1,7076,7,and,graphics


In [26]:
dict_index_t = OrderedDict((key, i) for i, key in enumerate(set(freq_df["term"].tolist())))
dict_index_c = OrderedDict((key, i) for i, key in enumerate(set(freq_df["col"].tolist())))

In [27]:
size_t = len(dict_index_t)
size_c = len(dict_index_c)
matrix_df = pd.DataFrame(index = dict_index_t.keys(), columns=dict_index_c.keys())

for k, f_tc in freq_df.iterrows():
	if f_tc.tolist()[1] >= 10:
		ppmi = calc_PPMI(f_tc.tolist())
		matrix_df.loc[f_tc[4], f_tc[5]] = ppmi

matrix_df.to_csv("./_re_84.tsv",sep="\t")#,index = False)

## 85. 主成分分析による次元圧縮

    84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．
	http://qiita.com/segavvy/items/f1a7f3200c3b771e8568

In [28]:
matrix_df = pd.read_csv("./_re_84.tsv",delimiter="\t").drop(0)

matrix_df.index = matrix_df.iloc[:,0]
del matrix_df["Unnamed: 0"]
matrix_df = matrix_df.fillna(0)
matrix_df

,coemgáin,limited,writings,systematic,Unnamed: 5,represent,dynamic,four,demanded,prefix,...,earthquake,rule,portion,compete,pupil,ideological,rural,50-bit,indirectly,nudists
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
limited,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
writings,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
folklore,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dynamic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
four,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
prefix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
asian,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hanging,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
consists,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
t_list = matrix_df.index
c_list = matrix_df.columns

clf = sklearn.decomposition.TruncatedSVD(300)
np_300 = clf.fit_transform(matrix_df)
vec_df = pd.DataFrame(np_300, index=matrix_df.index)
vec_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
limited,3.169457e-17,-9.385073e-15,6.232942e-15,-5.357332e-15,-4.135590e-15,6.682831e-15,-3.109714e-16,5.966837e-15,1.256554e-14,1.418172e-14,...,1.332703e-77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
writings,-1.499239e-20,-6.425486e-18,2.731105e-16,-7.970112e-15,-1.528635e-15,1.601135e-15,-3.583885e-15,9.053576e-15,-6.440303e-15,4.171820e-15,...,7.441349e-78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
folklore,-9.460857e-22,1.729781e-17,-1.960208e-15,-4.548071e-15,1.691925e-14,-4.816564e-15,2.591701e-15,1.109607e-15,-5.582437e-15,3.866283e-15,...,9.527762e-79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dynamic,9.388740e-21,-1.425773e-17,1.550746e-15,-3.297440e-15,-1.985069e-16,1.472996e-14,3.085708e-15,1.404884e-15,-7.475083e-15,-7.101098e-17,...,3.026247e-78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
four,-3.289663e-21,1.916916e-18,-3.602383e-16,-1.154342e-15,-2.038635e-15,-8.621160e-15,5.651239e-16,-2.184988e-14,8.127699e-15,-7.321900e-15,...,5.063364e-78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
prefix,2.199996e-21,-5.120349e-19,9.641281e-17,-3.547783e-16,1.813065e-15,3.927133e-15,3.835492e-15,2.916007e-15,-5.145687e-15,-5.332821e-16,...,-4.276076e-79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
asian,-1.827038e-21,1.084437e-18,-1.764396e-16,8.497349e-16,-2.746353e-15,-3.496464e-15,2.043823e-15,1.008906e-14,-4.330425e-15,-4.518177e-16,...,-9.679917e-78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hanging,4.035939e-23,4.008022e-20,-1.216135e-17,-5.323969e-17,5.883714e-17,-1.896854e-16,-1.100164e-16,3.340377e-16,-1.032755e-16,5.183263e-15,...,2.253338e-78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
consists,1.112692e-23,-5.867400e-20,5.506951e-18,-3.290466e-17,2.878531e-16,-1.306462e-16,-5.083850e-16,-6.303454e-16,7.717556e-16,2.738774e-15,...,-1.144267e-78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
vec_df.to_csv("./85.tsv",sep="\t")

### (保留) PCAとSVDの違い!

## 86. 単語ベクトルの表示

    85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [34]:
vec_df = pd.read_csv("./85.tsv",delimiter="\t",index_col = "Unnamed: 0")
vec_df.loc["asian"]

0     -1.827038e-21
1      1.084437e-18
2     -1.764396e-16
3      8.497349e-16
4     -2.746353e-15
5     -3.496464e-15
6      2.043823e-15
7      1.008906e-14
8     -4.330425e-15
9     -4.518177e-16
10    -2.780160e-15
11    -6.087199e-16
12    -3.214068e-15
13    -6.259518e-15
14    -1.216634e-15
15    -3.416939e-15
16     4.361643e-15
17    -6.706946e-17
18    -4.147236e-17
19    -2.839001e-15
20     1.733229e-15
21    -6.702783e-16
22    -1.756960e-15
23    -1.490783e-15
24     2.808490e-15
25     3.629803e-16
26    -1.358332e-15
27     1.464694e-16
28     9.186057e-16
29    -8.823861e-17
           ...     
270    0.000000e+00
271    0.000000e+00
272    0.000000e+00
273    0.000000e+00
274    2.252793e-78
275    0.000000e+00
276    0.000000e+00
277    0.000000e+00
278    0.000000e+00
279    0.000000e+00
280    0.000000e+00
281    0.000000e+00
282    0.000000e+00
283    0.000000e+00
284    0.000000e+00
285    0.000000e+00
286    0.000000e+00
287    0.000000e+00
288    0.000000e+00


## 87. 単語の類似度

    85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [37]:
vec_df = pd.read_csv("./85.tsv",delimiter="\t",index_col = "Unnamed: 0")
print vec_df

_tmp1 = vec_df.loc["paris"]
_tmp2 = vec_df.loc["asian"]
1-dis.cosine(_tmp1, _tmp2)

-0.011056801343120037

## 88. 類似度の高い単語10件

    85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [38]:
vec_df = pd.read_csv("./85.tsv",delimiter="\t",index_col = "Unnamed: 0")
tmp = vec_df.loc["limited"]

cosine_dic ={}
for i,row in vec_df.iterrows():
	cosine_dic.update({i:{"value":dis.cosine(tmp, row)}})

cosine_df = pd.DataFrame.from_dict(cosine_dic).T
cosine_df

,value
$1.10,NaN
"$10,000",NaN
$15.7,NaN
$160,NaN
$174,NaN
$2.74,NaN
$20,NaN
"$350,000",NaN
"$5,000",NaN
$5.76,NaN


## 89. 加法構成性によるアナロジー

    85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．


In [39]:
vec_df = pd.read_csv("./85.tsv",delimiter="\t",index_col = "Unnamed: 0")

In [44]:
_1_df = vec_df.loc["nippon"]
_2_df = vec_df.loc["tokyo"]
_3_df = vec_df.loc["paris"]
tmp_df = _1_df - _2_df + _3_df
tmp_df

0      4.405589e-26
1     -5.533249e-23
2      5.024956e-21
3      9.114564e-20
4     -7.684181e-21
5     -2.091186e-19
6      4.668823e-19
7      1.893097e-19
8      1.552141e-19
9      3.360414e-18
10    -5.769583e-19
11    -9.258879e-18
12     1.738424e-17
13     6.354169e-18
14    -2.964477e-17
15     2.498619e-17
16    -9.564575e-17
17    -7.035541e-17
18    -2.379738e-15
19     9.070296e-16
20    -4.921777e-15
21    -2.263100e-15
22     2.288679e-15
23     1.615057e-15
24     4.683462e-16
25     2.868686e-16
26     2.087963e-16
27     8.994581e-16
28     2.033311e-16
29     1.840807e-16
           ...     
270    0.000000e+00
271    0.000000e+00
272    0.000000e+00
273    0.000000e+00
274    1.709009e-78
275    0.000000e+00
276    0.000000e+00
277    0.000000e+00
278    0.000000e+00
279    0.000000e+00
280    0.000000e+00
281    0.000000e+00
282    0.000000e+00
283    0.000000e+00
284    0.000000e+00
285    0.000000e+00
286    0.000000e+00
287    0.000000e+00
288    0.000000e+00


In [46]:
cosine_dic ={}
for i,row in vec_df.iterrows():
	cosine_dic.update({i:{"value":1-dis.cosine(tmp_df, row)}})
cosine_df = pd.DataFrame.from_dict(cosine_dic).T
cosine_df.sort_values(by="value",ascending = False)[:11]

,value
paris,1.000000
schooling,0.153859
guiding,0.117827
operate,0.106944
dominate,0.065275
rise,0.046851
freethinkers,0.041729
exchangeable,0.039200
onfray,0.038811
anarchism,0.038572
